In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
# run bed to fasta - prepare for motif enrichment analysis
        
wkdir = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac"

hist_all = []
for i in range(5):
    f = wkdir + f'/fimo_L23A_M{i+1}/fimo.tsv'
    df = pd.read_csv(f, sep='\t').dropna()
    hist = df['motif_alt_id'].value_counts(ascending=False).to_frame(f'A{i+1}')
    hist_all.append(hist)
    
for i in range(3):
    f = wkdir + f'/fimo_L23B_M{i+1}/fimo.tsv'
    df = pd.read_csv(f, sep='\t').dropna()
    hist = df['motif_alt_id'].value_counts(ascending=False).to_frame(f'B{i+1}')
    hist_all.append(hist)
    
for i in range(5):
    f = wkdir + f'/fimo_L23C_M{i+1}/fimo.tsv'
    df = pd.read_csv(f, sep='\t').dropna()
    hist = df['motif_alt_id'].value_counts(ascending=False).to_frame(f'C{i+1}')
    hist_all.append(hist)

hist_all = pd.concat(hist_all, axis=1)
hist_all

In [ ]:
# for i, ax in enumerate(axs): 
ref = 'A1'
fig, axs = plt.subplots(3, 5, figsize=(4*5,4*3), sharex=True)
for i, tn in enumerate(['A', 'B', 'C']):
    for j in range(5):
        ax = axs[i,j]
        tst = f'{tn}{j+1}'
        if not tst in hist_all.columns:
            ax.axis('off')
            continue 
        
        ax.scatter(hist_all[ref], hist_all[tst], s=5)
        rankings = ((hist_all[tst]+100) / (hist_all[ref]+100)).sort_values(ascending=False) 
        rankings_top = rankings.head(3)
        ax.scatter(hist_all.loc[rankings_top.index][ref], 
                   hist_all.loc[rankings_top.index][tst], s=5)

        ax.set_title(f'{tst} vs {ref} \n{rankings_top.index.values}')

        ax.set_xscale('log')
        ax.set_yscale('log')

        sns.despine(ax=ax)

axs[0,0].set_xlabel('motif occurance')
axs[0,0].set_ylabel('motif occurance')
fig.tight_layout()
plt.show()